# Packages

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Data Loading and pre-processing

In [2]:
df = pd.read_csv('pause_freq_2_wide.csv', index_col='Filename')
df = df.drop(index='lslct035')
df = df.drop(index='lslct036')
df

,Discipline,Interactivity_by_bu_pause,Level,Class_size,Gender,between_clause,between_phrase,between_utterance,disfluency,event_related,within_phrase
Filename,,,,,,,,,,,
ahlct001,ah,medium,PG,medium,male,66.628809,52.432330,0.094643,22.146508,11.262540,17.508991
ahlct002,ah,low,UG,medium,male,42.553191,35.895209,0.144739,55.724417,4.921117,12.737010
ahlct003,ah,high,UG,small,male,73.476980,58.595567,2.945280,30.382886,5.270501,28.987754
ahlct004,ah,low,UG,medium,male,96.973347,95.919289,0.150580,9.486523,10.239422,35.536817
ahlct005,ah,low,UG,medium,male,70.723442,99.602181,0.000000,25.931929,6.482982,37.277147
...,...,...,...,...,...,...,...,...,...,...,...
sslct036,ss,medium,UG,small,male,55.114763,33.295551,0.141683,55.398130,23.377727,22.810995
sslct037,ss,high,UG,small,female,55.668016,21.761134,1.391700,21.002024,0.759109,14.929150
sslct038,ss,high,UG,large,female,54.283405,44.046336,2.963362,47.683190,23.437500,24.919181


In [3]:
df['Discipline'] = df['Discipline'].replace({'ah': 0, 
                          'ls': 1,
                          'ps': 2,
                          'ss': 3})

In [4]:
df.columns

Index(['Discipline', 'Interactivity_by_bu_pause', 'Level', 'Class_size',
       'Gender', 'between_clause', 'between_phrase', 'between_utterance',
       'disfluency', 'event_related', 'within_phrase'],
      dtype='object')

In [5]:
X = tf.cast(df.iloc[:, 9], dtype=tf.float32)/120
Y = tf.cast(df['Discipline'], dtype=tf.int64)

2021-07-23 22:54:46.050712: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
X.shape, Y.shape

(TensorShape([158]), TensorShape([158]))

# Model Building
## 1. relu + softmax

In [8]:
inputs = keras.Input(shape=(1,))
x = layers.Dense(4, activation=tf.nn.relu)(inputs)
for _ in range(4):
    x = layers.Dense(4, activation=tf.nn.relu)(x)
outputs = layers.Dense(4, activation=tf.nn.softmax)(x)

In [9]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 8         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 20    

In [11]:
# mini batch
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(X, Y, batch_size=64, epochs=50, validation_split=0.2)
train_scores = model.evaluate(X, Y, verbose=1)
print("Test loss:", train_scores[0])
print("Test accuracy:", train_scores[1])

Epoch 1/50
2/2 [==============================] - 1s 123ms/step - loss: 1.3362 - accuracy: 0.3175 - val_loss: 1.6346 - val_accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 18ms/step - loss: 1.3361 - accuracy: 0.3175 - val_loss: 1.6347 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 22ms/step - loss: 1.3360 - accuracy: 0.3175 - val_loss: 1.6348 - val_accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 21ms/step - loss: 1.3361 - accuracy: 0.3175 - val_loss: 1.6349 - val_accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 20ms/step - loss: 1.3360 - accuracy: 0.3175 - val_loss: 1.6350 - val_accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 18ms/step - loss: 1.3360 - accuracy: 0.3175 - val_loss: 1.6351 - val_accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 21ms/step - loss: 1.3360 - accuracy: 0.3175 - val_loss: 1.6352 - val_accuracy: 0.0000e+0

In [12]:
# batch

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(X, Y, epochs=50, validation_split=0.2)
train_scores = model.evaluate(X, Y, verbose=1)
print("Test loss:", train_scores[0])
print("Test accuracy:", train_scores[1])

Epoch 1/50
4/4 [==============================] - 1s 41ms/step - loss: 1.3355 - accuracy: 0.3175 - val_loss: 1.6384 - val_accuracy: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 0s 7ms/step - loss: 1.3356 - accuracy: 0.3175 - val_loss: 1.6386 - val_accuracy: 0.0000e+00
Epoch 3/50
4/4 [==============================] - 0s 7ms/step - loss: 1.3354 - accuracy: 0.3175 - val_loss: 1.6386 - val_accuracy: 0.0000e+00
Epoch 4/50
4/4 [==============================] - 0s 8ms/step - loss: 1.3353 - accuracy: 0.3175 - val_loss: 1.6387 - val_accuracy: 0.0000e+00
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 1.3353 - accuracy: 0.3175 - val_loss: 1.6388 - val_accuracy: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 1.3353 - accuracy: 0.3175 - val_loss: 1.6389 - val_accuracy: 0.0000e+00
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 1.3354 - accuracy: 0.3175 - val_loss: 1.6390 - val_accuracy: 0.0000e+00
Epoch

## 2. tanh + softmax

In [16]:
inputs2 = keras.Input(shape=(1,))
x2 = layers.Dense(4, activation=tf.nn.relu)(inputs2)
for _ in range(20):
    x2 = layers.Dense(4, activation=tf.nn.tanh)(x2)
outputs2 = layers.Dense(4, activation=tf.nn.softmax)(x2)

In [17]:
model2 = keras.Model(inputs=inputs2, outputs=outputs2)
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 8         
_________________________________________________________________
dense_29 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_32 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 20  

In [19]:
# mini batch

model2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model2.fit(X, Y, batch_size=64, epochs=50, validation_split=0.2)
train_scores = model2.evaluate(X, Y, verbose=1)
print("Test loss:", train_scores[0])
print("Test accuracy:", train_scores[1])

Epoch 1/50
2/2 [==============================] - 1s 193ms/step - loss: 1.3122 - accuracy: 0.3889 - val_loss: 1.6673 - val_accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 20ms/step - loss: 1.2851 - accuracy: 0.4365 - val_loss: 1.6596 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 20ms/step - loss: 1.2983 - accuracy: 0.4048 - val_loss: 1.6607 - val_accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 19ms/step - loss: 1.2778 - accuracy: 0.4683 - val_loss: 1.6651 - val_accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 24ms/step - loss: 1.2874 - accuracy: 0.4365 - val_loss: 1.6663 - val_accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 1.2701 - accuracy: 0.4921 - val_loss: 1.6706 - val_accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 1.2696 - accuracy: 0.4841 - val_loss: 1.6662 - val_accuracy: 0.0000e+0

In [21]:
# batch

model2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model2.fit(X, Y, batch_size=64, epochs=50, validation_split=0.2)
train_scores = model2.evaluate(X, Y, verbose=1)
print("Test loss:", train_scores[0])
print("Test accuracy:", train_scores[1])

Epoch 1/50
2/2 [==============================] - 1s 185ms/step - loss: 1.3045 - accuracy: 0.4048 - val_loss: 1.7120 - val_accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 22ms/step - loss: 1.2391 - accuracy: 0.5000 - val_loss: 1.6928 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 22ms/step - loss: 1.2755 - accuracy: 0.4524 - val_loss: 1.7033 - val_accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 1.2409 - accuracy: 0.5000 - val_loss: 1.6982 - val_accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 1.2442 - accuracy: 0.4841 - val_loss: 1.6968 - val_accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 21ms/step - loss: 1.2517 - accuracy: 0.4603 - val_loss: 1.6992 - val_accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 1.2380 - accuracy: 0.5079 - val_loss: 1.7110 - val_accuracy: 0.0000e+0